In [ ]:
# --- Standard Library ---
import os
import sys
import json
from datetime import timedelta

# --- Google Cloud Auth + APIs ---
from google.cloud import bigquery
from google.oauth2 import service_account
# from google.auth.transport.requests import AuthorizedSession

import gspread
from gspread_dataframe import set_with_dataframe

# --- Data & Visualization ---
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# --- Local Modules ---
from modules.pull_data import pull_and_append
from modules.flattening_json import flatten_extract_params, flatten_row, flatten_nested_column
from modules.column_order import column_order

In [5]:
# --- Configuration ---
SERVICE_ACCOUNT_KEY = './keys/key.json'
DATA_PATH = './data/data.json'
PROJECT_ID = "emojioracle-342f1"
DATASET_ID = "analytics_481352676"
SCOPES = [
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
# --- BigQuery Client ---
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY,
    scopes = SCOPES
)
bq_client = bigquery.Client(credentials = credentials, project = PROJECT_ID)

In [23]:
# will pull what is missing from BigQuery and merge into data_path
pull_and_append(credentials, PROJECT_ID, DATASET_ID, DATA_PATH)

Loaded existing data.
Latest event date in existing data: 20250418
No new data found or no tables to import.


In [24]:
pd.set_option('display.max_columns', None) # uncomment to see all of the cols in pandas dataframes

# load the data merged by pull_and_append into df
df = pd.read_json(DATA_PATH)

In [25]:
# flattenning

df = pd.DataFrame([flatten_row(row) for _, row in df.iterrows()]) # for wtfs refer to ./modules/flattening_json.py

In [ ]:
# cleaning & preprocessing

## dates and times
df = df.drop(columns=['event_date']) # built in case event_date may not be the same as the one in the event_timestamp

# convert unix time to ape-friendly datetime format
df['event_datetime'] = pd.to_datetime(df['event_timestamp'], unit='us', utc=True) 
df['event_previous_datetime'] = pd.to_datetime(df['event_previous_timestamp'], unit='us', utc=True)
df['event_first_touch_datetime'] = pd.to_datetime(df['user_first_touch_timestamp'], unit='us', utc=True)
df['user.first_open_datetime'] = pd.to_datetime(df['user.first_open_time'], unit='ms', utc=True)


df['event_date'] = df['event_datetime'].dt.date
df['event_time'] = df['event_datetime'].dt.time

df['event_previous_date'] = df['event_previous_datetime'].dt.date
df['event_previous_time'] = df['event_previous_datetime'].dt.time

df['event_first_touch_date'] = df['event_first_touch_datetime'].dt.date
df['event_first_touch_time'] = df['event_first_touch_datetime'].dt.time

df['user.first_open_date'] = df['user.first_open_datetime'].dt.date
df['user.first_open_time'] = df['user.first_open_datetime'].dt.time

df['device.time_zone_offset_hours'] = df['device.time_zone_offset_seconds'] / 3600 # seconds to hours
df['event_params.engagement_time_seconds'] = df['event_params.engagement_time_msec'] / 1000 # ms to seconds
df['event_server_delay_seconds'] = df['event_server_timestamp_offset'] / 1000 # ms to seconds 
df['event_params.time_spent_seconds'] = df['event_params.time_spent'] # just renaming for clarity

df = df.drop(columns=['event_timestamp',
                      'event_previous_timestamp', 
                      'user_first_touch_timestamp', 
                      'event_server_timestamp_offset', 
                      'device.time_zone_offset_seconds', 
                      'event_params.engagement_time_msec',
                      'event_datetime',
                      'event_previous_datetime',
                      'event_params.time_spent',
                      'event_first_touch_datetime',
                      'user.first_open_datetime'
                    ])



In [27]:
# I reordered the columns to make it easier to follow, in column_order.py

df = df[column_order]


df.columns = df.columns.str.replace('.', '__') # replace dots with dashes in column names bc of bigquerys nagging

In [33]:
df.tail(5)

,event_name,event_date,event_time,event_previous_date,event_previous_time,event_first_touch_date,event_first_touch_time,event_bundle_sequence_id,user_id,user_pseudo_id,user__first_open_date,user__first_open_time,user__ga_session_id,user__ga_session_number,app_info__id,app_info__firebase_app_id,app_info__version,app_info__install_store,app_info__install_source,device__advertising_id,device__vendor_id,device__category,device__mobile_brand_name,device__mobile_model_name,device__mobile_marketing_name,device__mobile_os_hardware_model,device__operating_system,device__operating_system_version,device__language,device__is_limited_ad_tracking,device__browser,device__browser_version,device__web_info,device__time_zone_offset_hours,geo__city,geo__country,geo__continent,geo__region,geo__sub_continent,geo__metro,traffic_source__name,traffic_source__medium,traffic_source__source,collected_traffic_source,event_params__ga_session_id,event_params__firebase_screen_id,event_params__ad_unit_id,event_params__ad_format,event_params__ad_network,event_params__ad_platform,event_params__ad_shown_where,event_params__answered_wrong,event_params__character_name,event_params__current_qi,event_params__current_tier,event_params__earned_amount,event_params__engaged_session_event,event_params__engagement_time_seconds,event_params__entrances,event_params__firebase_conversion,event_params__firebase_error,event_params__firebase_event_origin,event_params__firebase_screen_class,event_params__ga_session_number,event_params__how_its_earned,event_params__menu_name,event_params__mini_game_name,event_params__mini_game_ri,event_params__previous_first_open_count,event_params__session_engaged,event_params__spent_amount,event_params__spent_to,event_params__system_app,event_params__system_app_update,event_params__time_spent_seconds,event_params__update_with_analytics,event_params__where_its_earned,event_params__where_its_spent,event_params__currency_name,batch_event_index,batch_ordering_id,batch_page_id,privacy_info__ads_storage,privacy_info__analytics_storage,privacy_info__uses_transient_token,event_dimensions,event_server_delay_seconds,event_value_in_usd,ecommerce,is_active_user,platform,stream_id,user_ltv
15386,earn_virtual_currency,2025-04-18,19:22:10.191190,2025-04-18,19:21:54.046190,2025-04-09,09:32:18.415000,750,NaN,6bef18e3d26a07b8a20c26ff7eb26d82,2025-04-09,10:00:00,1.745001e+09,3.0,com.TestCompany.TestApp,1:332513280181:android:e9c9cb25732e9477377efc,0.1.0,None,com.google.android.packageinstaller,None,None,mobile,Xiaomi,MI 9,None,MI 9,Android,Android 11,en-us,No,None,None,None,3.0,Ankara,Türkiye,Asia,Ankara,Western Asia,(not set),(direct),(none),(direct),None,1745001270,7.667933e+17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,1.0,NaN,NaN,NaN,NaN,app,UnityPlayerActivity,3.0,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,question,NaN,gold,188,NaN,NaN,Yes,Yes,No,None,1289.674,NaN,None,True,ANDROID,10359646141,None
15387,menu_opened,2025-04-18,19:22:12.194191,2025-04-18,19:21:09.269191,2025-04-09,09:32:18.415000,750,NaN,6bef18e3d26a07b8a20c26ff7eb26d82,2025-04-09,10:00:00,1.745001e+09,3.0,com.TestCompany.TestApp,1:332513280181:android:e9c9cb25732e9477377efc,0.1.0,None,com.google.android.packageinstaller,None,None,mobile,Xiaomi,MI 9,None,MI 9,Android,Android 11,en-us,No,None,None,None,3.0,Ankara,Türkiye,Asia,Ankara,Western Asia,(not set),(direct),(none),(direct),None,1745001270,7.667933e+17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,app,UnityPlayerActivity,3.0,NaN,crystal_menu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189,NaN,NaN,Yes,Yes,No,None,1289.674,NaN,None,True,ANDROID,10359646141,None
15388,menu_opened,2025-04-18,19:22:12.679192,2025-04-18,19:22:12.194192,2025-04-09,09:32:18.415000,750,NaN,6bef18e3d26a07b8a20c26ff7eb26d82,2025-04-09,10:00:00,1.745001e+09,3.0,com.TestCompany.TestApp,1:332513280181:android:e9c9cb25732e9477377efc,0.1.0,None,com.google.android.packageinstaller,None,None,mobile,Xiaomi,MI 9,None,MI 9,Android,Android 11,en-us,

In [29]:
df.to_csv('./data/cleaned_data.csv', index=False) # save the cleaned data as a csv file

In [30]:
# Write to BigQuery (once, overwritin
table_id = f"{PROJECT_ID}.{DATASET_ID}.clean_data"

job = bq_client.load_table_from_dataframe(df, table_id, job_config=bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
))
job.result()  # Wait for job to finish

print(f"Cleaned data written to {table_id}. Ready to use in Looker.")

d:\GitHub\emoji-oracle-analytics\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
d:\GitHub\emoji-oracle-analytics\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:628: UserWarning: Pyarrow could not determine the type of columns: app_info__install_store, device__advertising_id, device__vendor_id, device__browser, device__browser_version, device__web_info, collected_traffic_source, event_dimensions, ecommerce, user_ltv.
  warnings.warn(


Cleaned data written to emojioracle-342f1.analytics_481352676.clean_data. Ready to use in Looker.


In [34]:
df.describe(include='all') # show the data types of the columns

,event_name,event_date,event_time,event_previous_date,event_previous_time,event_first_touch_date,event_first_touch_time,event_bundle_sequence_id,user_id,user_pseudo_id,user__first_open_date,user__first_open_time,user__ga_session_id,user__ga_session_number,app_info__id,app_info__firebase_app_id,app_info__version,app_info__install_store,app_info__install_source,device__advertising_id,device__vendor_id,device__category,device__mobile_brand_name,device__mobile_model_name,device__mobile_marketing_name,device__mobile_os_hardware_model,device__operating_system,device__operating_system_version,device__language,device__is_limited_ad_tracking,device__browser,device__browser_version,device__web_info,device__time_zone_offset_hours,geo__city,geo__country,geo__continent,geo__region,geo__sub_continent,geo__metro,traffic_source__name,traffic_source__medium,traffic_source__source,collected_traffic_source,event_params__ga_session_id,event_params__firebase_screen_id,event_params__ad_unit_id,event_params__ad_format,event_params__ad_network,event_params__ad_platform,event_params__ad_shown_where,event_params__answered_wrong,event_params__character_name,event_params__current_qi,event_params__current_tier,event_params__earned_amount,event_params__engaged_session_event,event_params__engagement_time_seconds,event_params__entrances,event_params__firebase_conversion,event_params__firebase_error,event_params__firebase_event_origin,event_params__firebase_screen_class,event_params__ga_session_number,event_params__how_its_earned,event_params__menu_name,event_params__mini_game_name,event_params__mini_game_ri,event_params__previous_first_open_count,event_params__session_engaged,event_params__spent_amount,event_params__spent_to,event_params__system_app,event_params__system_app_update,event_params__time_spent_seconds,event_params__update_with_analytics,event_params__where_its_earned,event_params__where_its_spent,event_params__currency_name,batch_event_index,batch_ordering_id,batch_page_id,privacy_info__ads_storage,privacy_info__analytics_storage,privacy_info__uses_transient_token,event_dimensions,event_server_delay_seconds,event_value_in_usd,ecommerce,is_active_user,platform,stream_id,user_ltv
count,15391,15391,15391,15110,15110,15391,15391,15391.000000,0.0,15391,15391,15391,1.536800e+04,15368.000000,15391,15391,15391,0,15391,0,0,15391,15362,15362,13845,15391,15391,15391,15391,15391,0,0,0,15391.000000,15391,15391,15391,15391,15391,15391,15391,15391,15391,0,1.539100e+04,1.525500e+04,396,159,159,396,396,739.000000,6356,6356,6356,3117.000000,15187.0,332.000000,92.0,2569.0,198.0,15391,15255,15368.000000,3117,3936,534,534,10.000000,88.0,446.000000,446,0.0,0.0,5352.000000,0.0,3117,446,3563,15391.000000,0.0,0.0,15391,15391,15391,0,15391.000000,0.0,0,15391,15391,1.539100e+04,0
unique,17,15,15391,15,15110,12,27,NaN,NaN,27,11,15,NaN,NaN,1,1,1,0,2,0,0,1,2,11,10,12,1,4,4,1,0,0,0,NaN,7,2,2,7,2,1,1,1,1,0,NaN,NaN,1,1,1,1,6,NaN,27,16,4,NaN,NaN,NaN,NaN,NaN,NaN,2,1,NaN,8,13,7,57,NaN,NaN,NaN,93,NaN,NaN,NaN,NaN,4,3,2,NaN,NaN,NaN,1,1,1,0,NaN,NaN,0,2,1,NaN,0
top,question_started,2025-04-06,19:22:38.960194,2025-04-06,19:22:37.285194,2025-04-04,21:10:44.888000,NaN,NaN,ddf2acb862daf1f5e1e9dff579ec87d7,2025-04-04,22:00:00,NaN,NaN,com.TestCompany.TestApp,1:332513280181:android:e9c9cb25732e9477377efc,0.1.0,NaN,com.google.android.packageinstaller,NaN,NaN,mobile,Samsung,SM-A525F,Galaxy A52 4G,SM-A525F,Android,Android 14,tr-tr,No,NaN,NaN,NaN,NaN,Ankara,Türkiye,Asia,Ankara,Western Asia,(not set),(direct),(none),(direct),NaN,NaN,NaN,test_unit_id,test_ad_format,test_ad_network,test_platform,wanna_play_ad,NaN,t,10,1,NaN,NaN,NaN,NaN,NaN,NaN,app,UnityPlayerActivity,NaN,combo,scroll_menu,stone_mini_game,stone_game,NaN,NaN,NaN,potion,NaN,NaN,NaN,NaN,question,board,gold,NaN,NaN,NaN,Yes,Yes,No,NaN,NaN,NaN,NaN,True,ANDROID,NaN,NaN
freq,3226,3589,1,3573,1,7866,7831,NaN,NaN,7831,7866,9035,NaN,NaN,15391,15391,15391,NaN,15362,NaN,NaN,15391,13546,11702,11702,11702,15391,12516,12896,15391,NaN,NaN,NaN,NaN,1400